In [1]:
!date

Tue Apr 27 11:45:40 PDT 2021


# Install packages and download data

In [7]:
import sys
COLAB = "google.colab" in sys.modules


In [1]:
if True:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 165 (delta 53), reused 121 (delta 23), pack-reused 0
Receiving objects: 100% (165/165), 59.24 MiB | 28.96 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [2]:
!pip --quiet install CITE-seq-Count==1.4.4

In [4]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.0 \
    anndata==0.7.6 \
    CITE-seq-Count==1.4.4

In [3]:
if Colab:
    # download the relevant data
    !wget -O cellhash_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/mpv37niu7kxbrx3gb4mfbun3t9zqv3ad.gz \
    2>&1
    !tar -xf cellhash_fastqs.tar.gz

cellhash_fastqs.tar 100%[===================>]   6.30G  17.0MB/s    in 6m 50s  


# Preprocess with kb

In [4]:
%%time
!INDEX="BMGP_2020/references/cellhash/kite/features.idx" && \
F2B="BMGP_2020/references/cellhash/kite/f2b.txt" && \
FASTA="BMGP_2020/references/cellhash/kite/features.fa" && \
FB="BMGP_2020/references/cellhash/kite/feature_barcodes.txt" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-05-18 12:20:15,395]    INFO Generating mismatch FASTA at BMGP_2020/references/cellhash/kite/features.fa
[2021-05-18 12:20:15,401]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/cellhash/kite/f2b.txt
[2021-05-18 12:20:15,404]    INFO Skipping kallisto index because BMGP_2020/references/cellhash/kite/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 24 ms, sys: 5.78 ms, total: 29.8 ms
Wall time: 1.66 s


In [1]:
%%bash
FB="BMGP_2020/references/cellhash/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/cellhash/kite/features.idx" && \
F2B="BMGP_2020/references/cellhash/kite/f2b.txt" && \
FASTA="BMGP_2020/references/cellhash/kite/features.fa" && \
OUT="kite_out" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
kb count \
-i $INDEX \
-g $F2B \
-x 10xv2 \
-o $OUT \
--workflow kite \
--h5ad \
--filter bustools \
--overwrite \
$FASTQS

[2021-06-11 09:36:09,676]    INFO Using index BMGP_2020/references/cellhash/kite/features.idx to generate BUS file to kite_out from
[2021-06-11 09:36:09,676]    INFO         fastqs/SRR8281307_R1.fastq.gz
[2021-06-11 09:36:09,677]    INFO         fastqs/SRR8281307_R2.fastq.gz
[2021-06-11 09:38:48,210]    INFO Sorting BUS file kite_out/output.bus to kite_out/tmp/output.s.bus
[2021-06-11 09:39:09,130]    INFO Whitelist not provided
[2021-06-11 09:39:09,131]    INFO Copying pre-packaged 10XV2 whitelist to kite_out
[2021-06-11 09:39:09,222]    INFO Inspecting BUS file kite_out/tmp/output.s.bus
[2021-06-11 09:39:14,225]    INFO Correcting BUS records in kite_out/tmp/output.s.bus to kite_out/tmp/output.s.c.bus with whitelist kite_out/10xv2_whitelist.txt
[2021-06-11 09:39:19,009]    INFO Sorting BUS file kite_out/tmp/output.s.c.bus to kite_out/output.unfiltered.bus
[2021-06-11 09:39:28,177]    INFO Generating count matrix kite_out/counts_unfiltered/cells_x_features from BUS file kite_out/outpu

In [2]:
print("hello")

hello


# Preprocess with cite-seq-count

In [3]:
!gunzip BMGP_2020/references/whitelist_10xv2.txt.gz

In [ ]:
!R1="fastqs/SRR8281307_R1.fastq.gz" && \
R2="fastqs/SRR8281307_R2.fastq.gz" && \
FB="BMGP_2020/references/cellhash/cite_seq_barcodes.csv" && \
WL="BMGP_2020/references/whitelist_10xv2.txt" && \
/usr/bin/time -v CITE-seq-Count -R1 $R1  -R2 $R2 --no_umi_correction -t $FB -wl $WL -cbf 1 -cbl 16 -umif 17 -umil 26 -cells 65000 -o ./cite_seq_out

Loading whitelist
[WARNING] Read1 length is 29bp but you are using 26bp for Cell and UMI barcodes combined.
This might lead to wrong cell attribution and skewed umi counts.

Counting number of reads
Started mapping
Processing 74,219,921 reads
CITE-seq-Count is running with 48 cores.
Processed 1,000,000 reads in 12.52 seconds. Total reads: 1,000,000 in child 19821
Mapping done for process 19821. Processed 1,546,248 reads
Processed 1,000,000 reads in 20.1 seconds. Total reads: 1,000,000 in child 19822
Mapping done for process 19822. Processed 1,546,248 reads
Processed 1,000,000 reads in 26.82 seconds. Total reads: 1,000,000 in child 19823
Mapping done for process 19823. Processed 1,546,248 reads
Processed 1,000,000 reads in 34.09 seconds. Total reads: 1,000,000 in child 19824
Mapping done for process 19824. Processed 1,546,248 reads
Processed 1,000,000 reads in 43.32 seconds. Total reads: 1,000,000 in child 19825
Mapping done for process 19825. Processed 1,546,248 reads
Processed 1,000,0